### Arshia Noor Project 3 Intro to DL

### Importing Packages

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as pyplot

import os
import pathlib
import tensorflow as tf
from PIL import Image
from skimage.transform import resize
from random import shuffle

from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications

### Uploading the dataset in train and test

In [28]:
batch_size = 64
img_height = 256
img_width = 256
ds_pet_dir = r'C:\Users\HP\Downloads\PetImages'
ds_pet = tf.keras.preprocessing.image_dataset_from_directory(r'C:\Users\HP\Downloads\PetImages')

Found 3911 files belonging to 2 classes.


In [29]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    ds_pet_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 3911 files belonging to 2 classes.
Using 3129 files for training.


In [31]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    ds_pet_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 3911 files belonging to 2 classes.
Using 782 files for validation.


### Part 1

In [46]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=5, kernel_size=3, activation='relu', input_shape=[256, 256, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten()
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='relu'))
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [65]:
cnn.fit( train_ds, validation_data = val_ds, epochs = 10)

Epoch 1/10
49/49 [==============================] - 91s 2s/step - loss: 11.9986 - accuracy: 0.2132 - val_loss: 11.9342 - val_accuracy: 0.2174
Epoch 2/10
49/49 [==============================] - 91s 2s/step - loss: 11.9986 - accuracy: 0.2132 - val_loss: 11.9342 - val_accuracy: 0.2174
Epoch 3/10
49/49 [==============================] - 88s 2s/step - loss: 11.9986 - accuracy: 0.2132 - val_loss: 11.9342 - val_accuracy: 0.2174
Epoch 4/10
49/49 [==============================] - 87s 2s/step - loss: 11.9986 - accuracy: 0.2132 - val_loss: 11.9342 - val_accuracy: 0.2174
Epoch 5/10
49/49 [==============================] - 90s 2s/step - loss: 11.9986 - accuracy: 0.2132 - val_loss: 11.9342 - val_accuracy: 0.2174
Epoch 6/10
49/49 [==============================] - 92s 2s/step - loss: 11.9986 - accuracy: 0.2132 - val_loss: 11.9342 - val_accuracy: 0.2174
Epoch 7/10
49/49 [==============================] - 96s 2s/step - loss: 11.9986 - accuracy: 0.2132 - val_loss: 11.9342 - val_accuracy: 0.2174
Epoch 

In [25]:
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 5)         140       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 5)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 32)        1472      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

### Part 2

In [66]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [67]:
pre_trained_model = InceptionV3(input_shape =(150, 150, 3), # Shape of our images 
                                 include_top= False, # Leave out the last fully connected layer
                                 weights = 'imagenet')

In [68]:
for layer in pre_trained_model.layers: layer.trainable = False

In [72]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [99]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (150, 150))
# Flow validation images in batches of 20 using test_datagen generator 
validation_generator = test_datagen.flow_from_directory( valid_dir,batch_size = 20,
                                                        class_mode ='binary', 
                                                        target_size= (150, 150))

Found 3617 images belonging to 2 classes.
Found 1268 images belonging to 2 classes.


In [102]:
last_layer = pre_trained_model.get_layer('mixed10')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 3, 3, 2048)


In [103]:
from tensorflow.keras.optimizers import RMSprop
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)

In [104]:
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

pretrain_model = Model(pre_trained_model.input, x) 

In [105]:
pretrain_model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

In [106]:
pretrain_history =pretrain_model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 5,
            validation_steps = 50,
            verbose = 2)

Epoch 1/5
100/100 - 118s - loss: 0.7923 - acc: 0.8818 - val_loss: 0.1011 - val_acc: 0.9590 - 118s/epoch - 1s/step
Epoch 2/5
100/100 - 118s - loss: 0.2676 - acc: 0.9194 - val_loss: 0.1021 - val_acc: 0.9690 - 118s/epoch - 1s/step
Epoch 3/5
100/100 - 111s - loss: 0.2268 - acc: 0.9295 - val_loss: 0.0892 - val_acc: 0.9760 - 111s/epoch - 1s/step
Epoch 4/5
100/100 - 114s - loss: 0.2924 - acc: 0.9265 - val_loss: 0.0867 - val_acc: 0.9780 - 114s/epoch - 1s/step
Epoch 5/5
100/100 - 114s - loss: 0.2226 - acc: 0.9414 - val_loss: 0.0725 - val_acc: 0.9770 - 114s/epoch - 1s/step


In [108]:
pretrain_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_192 (Conv2D)            (None, 74, 74, 32)   864         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_188 (Batch  (None, 74, 74, 32)  96          ['conv2d_192[0][0]']             
 Normalization)                                                                                   
                                                                                            